In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import joblib
from time import time
from datetime import datetime

In [2]:
tiempo_inicio = time()
today = f'[{datetime.today().strftime("%Y-%m-%d %H:%M:%S")}]'
print(f'{today} Comienzo de la predicción')

[2023-04-11 13:05:04] Comienzo de la predicción


In [3]:
# Variables Globales
PATH = os.getcwd()
AUX_FOLDER = 'aux_file'
OUTPUT = 'output'
MODEL_PARAMETERS = 'model_weights.pkl'
OUTPUT_FILENAME = 'predictions.xlsx'

In [4]:
# Si no esta la carpeta aux_file, la creo
if not os.path.exists(os.path.join(PATH, OUTPUT)):
    os.mkdir(OUTPUT)

In [5]:
new_data = {
    'Fecha': ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01'],
    'Area': [100, 200, 150, 120, 180],
    'Habitaciones': [2, 3, 4, 3, 4],
    'Antiguedad': [10, 20, 30, 40, 50],
}

new_df = pd.DataFrame(new_data)

In [6]:
media = 0
desv_estandar = 1

In [7]:
# Parameters
media = [127.3342857142857, 3.125714285714286, 50.62285714285714, 327124.0115305251]
desv_estandar = [
    42.55987005120564,
    1.402516688394291,
    27.88344194488689,
    142067.1613139045,
]


In [8]:
# Aplicar el mismo preprocesamiento que al dataset original
scaled_new_data = (new_df[['Area', 'Habitaciones', 'Antiguedad']] - media[:-1]) / desv_estandar[:-1]
new_df_scaled = pd.DataFrame(scaled_new_data, columns=['Area', 'Habitaciones', 'Antiguedad'])

# Cargar el modelo previously trained desde su archivo "model_weights.plk"
coeficientes, intercept = joblib.load(os.path.join(PATH, AUX_FOLDER, MODEL_PARAMETERS))

# Realizar predicciones con el modelo entrenado
predictions =  np.dot(new_df_scaled, coeficientes.reshape(-1,1)) + intercept

# Escalo el resultado
predictions = (predictions * desv_estandar[-1]) + media[-1]

# Guardar las predicciones en un archivo de Excel
predictions_df = pd.DataFrame(predictions, columns=['Predicciones'])
predictions_df = pd.concat([new_df, predictions_df], axis=1)
predictions_df.to_excel(os.path.join(PATH, OUTPUT, OUTPUT_FILENAME), index=False)

In [9]:
tiempo_fin = time()
duracion = tiempo_fin - tiempo_inicio
horas, remainder = divmod(int(duracion), 3600)
minutos, segundos = divmod(remainder, 60)
print(f'{today} El tiempo transcurrido de ejecucion fue: {horas:02d}h {minutos:02d}m {segundos:02d}s')

[2023-04-11 13:05:04] El tiempo transcurrido de ejecucion fue: 00h 00m 00s


In [10]:
coeficientes

array([ 0.89828297,  0.14628686, -0.39102177])

In [11]:
intercept

1.0435113261461618e-16

In [12]:
predictions_df

,Fecha,Area,Habitaciones,Antiguedad,Predicciones
0,2021-01-01,100,2,10,309412.502842
1,2021-02-01,200,3,20,604159.589655
2,2021-03-01,150,4,30,449129.063826
3,2021-04-01,120,3,40,324432.790440
4,2021-05-01,180,4,50,499239.180548
